In [ ]:
!nvidia-smi

Thu Dec 18 16:10:34 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   35C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
!pip install -U \
  transformers \
  datasets \
  accelerate \
  peft \
  bitsandbytes \
  trl

In [ ]:
!pip install evaluate rouge_score sacrebleu bert_score

In [ ]:
from datasets import load_dataset

dataset = load_dataset(
    "json",
    data_files="/content/drive/MyDrive/LLM_Final_Project/afet_llm_final_dataset.jsonl",
    split="train"
)

print(dataset)
print("Toplam örnek:", len(dataset))
print("İlk örnek:\n", dataset[0])

Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 400
})
Toplam örnek: 400
İlk örnek:
 {'instruction': 'Deprem sonrası iletişim tamamen kesildiğinde enkaz altında kalan yaralıya ilk yaklaşım nasıl olmalıdır?', 'input': 'Telsizler çalışmıyor, GSM yok, enkazdan zayıf ses geliyor.', 'output': 'Önce ekip güvenliği sağlanır. Artçı, gaz ve elektrik riski kontrol edilir. Yaralıya sözlü olarak güven verilir. Ses yanıtı alınamazsa dokunsal sinyal denenir. Enkaz oynatılmadan hava yolu değerlendirilir ve durum ekip üyelerine ışık veya el işaretleriyle bildirilir.'}


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_name = "Qwen/Qwen2-1.5B-Instruct"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    trust_remote_code=True
)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

In [ ]:
from peft import LoraConfig, get_peft_model
import torch

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()


trainable params: 2,179,072 || all params: 1,545,893,376 || trainable%: 0.1410


In [ ]:
from transformers import Trainer, TrainingArguments

def format_example(example):
    return f"""### Talimat:
{example['instruction']}

### Girdi:
{example['input']}

### Yanıt:
{example['output']}"""

def tokenize_function(example):
    text = format_example(example)

    tokenized = tokenizer(
        text,
        truncation=True,
        max_length=1024,
        padding="max_length"
    )

    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

tokenized_dataset = dataset.map(tokenize_function, remove_columns=dataset.column_names)

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/LLM_Final_Project/afetllm_lora",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    num_train_epochs=3,
    fp16=True,
    logging_steps=10,
    save_steps=100,
    save_total_limit=2,
    report_to="none"
)

In [ ]:
import torch
torch.cuda.empty_cache()


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset
)

trainer.train()

Step,Training Loss
10,8.083000
20,0.721600
30,0.264500
40,0.219800
50,0.207500
60,0.187300
70,0.182100
80,0.177100
90,0.163300
100,0.172800


TrainOutput(global_step=300, training_loss=0.44524634957313536, metrics={'train_runtime': 645.4705, 'train_samples_per_second': 1.859, 'train_steps_per_second': 0.465, 'total_flos': 9676945096704000.0, 'train_loss': 0.44524634957313536, 'epoch': 3.0})

In [ ]:
from peft import PeftModel
from transformers import AutoModelForCausalLM

base_model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen2-1.5B-Instruct",
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

model = PeftModel.from_pretrained(
    base_model,
    "/content/drive/MyDrive/LLM_Final_Project/afetllm_lora/checkpoint-300"
)

model.eval()


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen2ForCausalLM(
      (model): Qwen2Model(
        (embed_tokens): Embedding(151936, 1536)
        (layers): ModuleList(
          (0-27): 28 x Qwen2DecoderLayer(
            (self_attn): Qwen2Attention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=1536, out_features=1536, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=1536, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=1536, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): Line

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(
    "Qwen/Qwen2-1.5B-Instruct",
    trust_remote_code=True
)
tokenizer.pad_token = tokenizer.eos_token


In [ ]:
prompt = """### Talimat:
Deprem sonrası iletişimin tamamen kesildiği bir ortamda yaralı tahliyesi nasıl planlanmalıdır?

### Girdi:
GSM ve telsiz yok, artçı sarsıntılar devam ediyor.

### Yanıt:
"""


In [ ]:
inputs = tokenizer(
    prompt,
    return_tensors="pt"
).to("cuda")


In [ ]:
import torch

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=200,
        do_sample=True,
        temperature=0.7,
        top_p=0.9
    )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))


### Talimat:
Deprem sonrası iletişimin tamamen kesildiği bir ortamda yaralı tahliyesi nasıl planlanmalıdır?

### Girdi:
GSM ve telsiz yok, artçı sarsıntılar devam ediyor.

### Yanıt:
Tahliye planı genel bir yasallaştırma ile sağlanır. İletişim olmayan bölgelerde herhangi bir temsili yapmaz.


In [ ]:
with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=200,
        do_sample=False,      # ← önemli
        temperature=0.0       # ← önemli
    )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


### Talimat:
Deprem sonrası iletişimin tamamen kesildiği bir ortamda yaralı tahliyesi nasıl planlanmalıdır?

### Girdi:
GSM ve telsiz yok, artçı sarsıntılar devam ediyor.

### Yanıt:
Tahliye planı genel kitleye açık olarak sunulmalı. Yaralıların güvenliği ve bilgi sağlamlığı önemli.


In [ ]:
base_model.eval()

with torch.no_grad():
    outputs = base_model.generate(
        **inputs,
        max_new_tokens=200
    )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))


### Talimat:
Deprem sonrası iletişimin tamamen kesildiği bir ortamda yaralı tahliyesi nasıl planlanmalıdır?

### Girdi:
GSM ve telsiz yok, artçı sarsıntılar devam ediyor.

### Yanıt:
Yaralıların temsilci olarak belirli etkinlikleri takip etmesini önerilir. Yaralıların hareket planını oluşturmada bilgiye erişim sağlanır.


In [ ]:
sop_prompt = """### Talimat:
Deprem sonrası iletişimin tamamen kesildiği bir ortamda, AFAD benzeri resmi
arama-kurtarma prosedürlerine göre yaralı tahliyesi nasıl planlanmalıdır?

### Girdi:
GSM, telsiz ve internet yok.
Artçı sarsıntılar devam ediyor.
Binada gaz ve yapısal hasar riski mevcut.
Birden fazla yaralı var.

### Yanıt:
"""


In [ ]:
base_model.eval()

inputs = tokenizer(
    sop_prompt,
    return_tensors="pt"
).to(base_model.device)

with torch.no_grad():
    output_base = base_model.generate(
        **inputs,
        max_new_tokens=200
    )

print("=== BASE MODEL ÇIKTISI ===")
print(tokenizer.decode(output_base[0], skip_special_tokens=True))


=== BASE MODEL ÇIKTISI ===
### Talimat:
Deprem sonrası iletişimin tamamen kesildiği bir ortamda, AFAD benzeri resmi
arama-kurtarma prosedürlerine göre yaralı tahliyesi nasıl planlanmalıdır?

### Girdi:
GSM, telsiz ve internet yok.
Artçı sarsıntılar devam ediyor.
Binada gaz ve yapısal hasar riski mevcut.
Birden fazla yaralı var.

### Yanıt:
Yaralıler 30 dakika sürenin başlangıcını belirlemektedir. Yerel kendi koruyucular tarafından belirlenen temel alanlar, korkunmaz veya hafif zorlukları içerir.


In [ ]:
model.eval()

inputs = tokenizer(
    sop_prompt,
    return_tensors="pt"
).to(model.device)

with torch.no_grad():
    output_ft = model.generate(
        **inputs,
        max_new_tokens=200
    )

print("=== FINE-TUNED MODEL ÇIKTISI ===")
print(tokenizer.decode(output_ft[0], skip_special_tokens=True))


=== FINE-TUNED MODEL ÇIKTISI ===
### Talimat:
Deprem sonrası iletişimin tamamen kesildiği bir ortamda, AFAD benzeri resmi
arama-kurtarma prosedürlerine göre yaralı tahliyesi nasıl planlanmalıdır?

### Girdi:
GSM, telsiz ve internet yok.
Artçı sarsıntılar devam ediyor.
Binada gaz ve yapısal hasar riski mevcut.
Birden fazla yaralı var.

### Yanıt:
Yaralıların durumları ve çevresel durumu bilinirken, yaralıların güvenliği korunmalı.


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel

# 1. Yapılandırmayı Yeniden Tanımla (Eksik olan buydu)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

model_name = "Qwen/Qwen2-1.5B-Instruct"

# 2. Base Modeli Yükle
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

# 3. Kendi Eğittiğin LoRA Ağırlıklarını Yükle
# Klasör yolunun doğru olduğundan emin ol (checkpoint-300 veya en sonuncusu)
model = PeftModel.from_pretrained(
    base_model,
    "/content/drive/MyDrive/LLM_Final_Project/afetllm_lora/checkpoint-300"
)

# 4. Tokenizer'ı Yükle
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

model.eval()
print("Model başarıyla yüklendi, değerlendirmeye hazır!")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Model başarıyla yüklendi, değerlendirmeye hazır!


In [ ]:
from datasets import load_dataset

# Veriyi yükle
dataset = load_dataset(
    "json",
    data_files="/content/drive/MyDrive/LLM_Final_Project/afet_llm_final_dataset.jsonl",
    split="train"
)

# Veriyi %10 test olacak şekilde böl (seed=42 ile her zaman aynı örnekleri seçeriz)
dataset_split = dataset.train_test_split(test_size=0.1, seed=42)
test_dataset = dataset_split["test"]

print(f"Test edilecek örnek sayısı: {len(test_dataset)}")

Generating train split: 0 examples [00:00, ? examples/s]

Test edilecek örnek sayısı: 40


In [ ]:
import evaluate
import torch
from tqdm import tqdm
import numpy as np

# Metrikleri yükle
rouge = evaluate.load("rouge")
bleu = evaluate.load("sacrebleu")

predictions = []
references = []

print("Model yanıt üretiyor, lütfen bekleyin...")

# Test setindeki her örnek için tahmin üret
for example in tqdm(test_dataset):
    # Eğitim formatına uygun prompt oluştur
    prompt = f"### Talimat:\n{example['instruction']}\n\n### Girdi:\n{example['input']}\n\n### Yanıt:\n"

    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=150,
            do_sample=False, # Skor ölçerken rastgeleliği kapatıyoruz
            temperature=0.0
        )

    full_output = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Sadece modelin ürettiği yanıt kısmını (### Yanıt'tan sonrasını) ayır
    if "### Yanıt:" in full_output:
        pred_response = full_output.split("### Yanıt:")[1].strip()
    else:
        pred_response = full_output.strip()

    predictions.append(pred_response)
    references.append(example['output'])

# --- HESAPLAMALAR ---

# 1. ROUGE Skorları
rouge_results = rouge.compute(predictions=predictions, references=references)

# 2. BLEU Skoru
bleu_results = bleu.compute(predictions=predictions, references=[[r] for r in references])

# 3. Kelime Bazlı F1 Skoru (Basit mantıkla)
def calculate_f1(pred, ref):
    p_tokens = set(pred.lower().split())
    r_tokens = set(ref.lower().split())
    if not p_tokens or not r_tokens: return 0.0
    common = p_tokens.intersection(r_tokens)
    if not common: return 0.0
    prec = len(common) / len(p_tokens)
    rec = len(common) / len(r_tokens)
    return 2 * (prec * rec) / (prec + rec)

f1_scores = [calculate_f1(p, r) for p, r in zip(predictions, references)]

print("\n" + "="*30)
print("NİCEL DEĞERLENDİRME SONUÇLARI")
print("="*30)
print(f"ROUGE-1: {rouge_results['rouge1']:.4f}")
print(f"ROUGE-L: {rouge_results['rougeL']:.4f}")
print(f"BLEU Skoru: {bleu_results['score']:.2f}")
print(f"Ortalama F1 Skoru: {np.mean(f1_scores):.4f}")
print("="*30)

Model yanıt üretiyor, lütfen bekleyin...


100%|██████████| 40/40 [02:35<00:00,  3.88s/it]



NİCEL DEĞERLENDİRME SONUÇLARI
ROUGE-1: 0.2023
ROUGE-L: 0.1723
BLEU Skoru: 2.28
Ortalama F1 Skoru: 0.1202
